In [0]:
# Exemplo de registro de modelo e listagem com tags específicas

In [0]:
%pip install ultralytics

INFO: pip is looking at multiple versions of contourpy to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 152.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 139.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 123.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 142.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 152.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 166.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 

In [0]:
import base64
from PIL import Image
import io
import requests
import pandas as pd

In [0]:
# Carregue o modelo pré-treinado


import logging
logging.getLogger().setLevel(logging.WARNING)

model = YOLO("yolo11s.pt")



from pydantic import BaseModel, Json
from typing import List, Union, Any, Dict
import mlflow


class Input(pydantic.BaseModel):
    image_base64: str

# Crie um wrapper
class YoloWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        from ultralytics import YOLO
        self.model = YOLO(context.artifacts["model_path"])

    def predict(self, context, model_input: pd.DataFrame) -> pd.DataFrame:
        import base64, io
        from PIL import Image
        import numpy as np

        b64 = model_input.iloc[0]["image_base64"]
        img_bytes = base64.b64decode(b64)
        image = Image.open(io.BytesIO(img_bytes)).convert("RGB")
        image_np = np.array(image)
        results = self.model(image_np)
        output_buffer = io.BytesIO()
        annotated_img = results[0].plot() if hasattr(results[0], "plot") else results[0].imgs[0]
        annotated_pil = Image.fromarray(annotated_img)
        annotated_pil.save(output_buffer, format="JPEG")
        output_b64 = base64.b64encode(output_buffer.getvalue()).decode("utf-8")
        return pd.DataFrame({"predicted_image_base64": [output_b64]})


# Crie um arquivo temporário
temp_path = "/tmp/yolo11s.pt"
model.save(temp_path)

# Crie um input de exemplo
img_url = "https://cdn.pixabay.com/photo/2020/04/09/10/47/car-5020870_1280.jpg"

# Baixe e codifique a imagem em base64
img_response = requests.get(img_url)
b64_img = base64.b64encode(img_response.content).decode("utf-8")

input_example = pd.DataFrame({
    "image_base64": [b64_img]
})

# Exemplo de saída para signature
predicted_example = pd.DataFrame({"predicted_image_base64": ["exemplo_base64"]})
signature = infer_signature(input_example, predicted_example)

with mlflow.start_run() as run:
    run = mlflow.active_run()
    run_id = run.info.run_id
    mlflow.pyfunc.log_model(
        artifact_path="model", 
        python_model=YoloWrapper(),
        artifacts={"model_path": temp_path},
        pip_requirements=["ultralytics", "pillow", "numpy"],
        input_example=input_example,
        signature=signature
    )


2025/09/24 21:14:50 INFO mlflow.pyfunc: Validating input example against model signature



0: 448x640 1 person, 2 cars, 2 potted plants, 237.1ms
Speed: 2.4ms preprocess, 237.1ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)


In [0]:
run_id

'9cd1b0e342dd4b688155df284dcee1e8'

In [0]:
run = mlflow.active_run()
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri, "baraldi_catalog.genai.yolo_model")

Successfully registered model 'baraldi_catalog.genai.yolo_model'.
Created version '1' of model 'baraldi_catalog.genai.yolo_model'.


<ModelVersion: aliases=[], creation_timestamp=1758748720019, current_stage=None, description='', last_updated_timestamp=1758748721873, name='baraldi_catalog.genai.yolo_model', run_id='9cd1b0e342dd4b688155df284dcee1e8', run_link=None, source='dbfs:/databricks/mlflow-tracking/2964497923820417/9cd1b0e342dd4b688155df284dcee1e8/artifacts/model', status='READY', status_message='', tags={}, user_id='daniel.baraldi@databricks.com', version='1'>

In [0]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
client.set_registered_model_tag("baraldi_catalog.genai.yolo_model", "tag_env", "producao")

In [0]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
client.set_registered_model_tag("baraldi_catalog.genai.yolo_model", "tag_departamento", "RH")

In [0]:
client.set_model_version_tag(
    name=f"{catalog}.{schema}.yolo_model",  
    version="1",                              
    key="validation_status",                  
    value="approved"                          
)

In [0]:
w.registered_models.set_alias(
    full_name=f"{catalog}.{schema}.yolo_model",
    alias="Champion",
    version_num=1
)

RegisteredModelAlias(alias_name='champion', version_num=1)

In [0]:
w.registered_models.set_alias(
    full_name=f"{catalog}.{schema}.yolo_model",
    alias="Champion",
    version_num=1
)

In [0]:
catalog="baraldi_catalog"
schema="genai"
from databricks.sdk import WorkspaceClient
import json

w = WorkspaceClient()
model_list = list(w.registered_models.list(catalog_name=catalog, schema_name=schema))
for model in model_list:
    model_name = model.name
    details = client.get_registered_model(f"{catalog}.{schema}.{model_name}")
    versions=w.model_versions.list(full_name=f"{catalog}.{schema}.{model_name}")
    dict_modelos = {
        f"{catalog}.{schema}.{model_name}": [
            {
                "tags": details.tags,
                "version": [
                    {
                        str(version.version): [
                            client.get_model_version(f"{catalog}.{schema}.{model_name}", str(version.version)).tags
                        ]
                    } for version in versions
                ]
            }
        ]
    }
print(json.dumps(dict_modelos,indent=2))
#print(json.dumps(dict_modelos["baraldi_catalog.genai.yolo_model"],indent=2))

{
  "baraldi_catalog.genai.yolo_model": [
    {
      "tags": {
        "tag_departamento": "RH",
        "tag_env": "producao"
      },
      "version": [
        {
          "1": [
            {
              "validation_status": "approved"
            }
          ]
        }
      ]
    }
  ]
}


In [0]:
# Filtrando os modelos que estão em producao
for nome_modelo, entries in dict_modelos.items():
    for entry in entries:
        if entry.get("tags", {}).get("tag_env") == "producao":
            print(nome_modelo)

baraldi_catalog.genai.yolo_model


In [0]:
# Filtrando apenas a versão que está aprovada
for entries in dict_modelos.values():
    for entry in entries:
        for versao in entry.get("version", []):
            for vnum, vtags_list in versao.items():
                for vtags in vtags_list:
                    if vtags.get("validation_status") == "approved":
                        print(vnum)

1


In [0]:
#Fazendo o download da versão aprovada e do modelo em prod
mlflow.set_registry_uri("databricks-uc")

destination_path = "./meumodelo_v1"

model_uri = f"models:/{nome_modelo}/{vnum}"

mlflow.artifacts.download_artifacts(
    artifact_uri=model_uri,
    dst_path=destination_path
)

'/Workspace/Users/daniel.baraldi@databricks.com/LaboratorioDatabricks/meumodelo_v1/'